In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN, GAT, GCNSVD, GCNJaccard, RGCN, ProGNN, SimPGCN
from deeprobust.graph.global_attack import Metattack

In [2]:
import scanpy as sc
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN, GAT
from deeprobust.graph.global_attack import Metattack

In [3]:
import deeprobust

In [4]:
deeprobust

<module 'deeprobust' from '/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/deeprobust/__init__.py'>

In [3]:
seq_data =sc.read_h5ad("/gpfs/gibbs/pi/zhao/yl2687/data/spatial_fish_data/xenium_breast/spe_xenium_withMetrics.h5ad")
seq_data.obs['celltype'] = seq_data.obs['scClassify'].copy()
seq_data.var_names_make_unique()
adata = seq_data

In [4]:
sc.pp.subsample(adata, random_state=2023, fraction=0.05)

adata.X = adata.layers['logcounts']

adata.obsm['spatial'] = adata.obsm['coord']

In [5]:
adata.obs['cluster'] = adata.obs['celltype'].copy()

In [6]:
import sklearn.model_selection
import sklearn.preprocessing

In [7]:
train_obs, test_obs =sklearn.model_selection.train_test_split(range(len(adata)), test_size=0.33, random_state=2023)

In [8]:
adata_train = adata[train_obs]

In [30]:
# train_edges = extract_edge_index(adata_train)
sc.pp.neighbors(adata, use_rep = 'spatial', n_neighbors=num_neig)

In [31]:
# train_edges
num_neig = 10

In [32]:
le = sklearn.preprocessing.LabelEncoder()

In [33]:
labels = torch.FloatTensor(le.fit_transform(adata.obs['celltype'])).long()

In [34]:
labels

tensor([14,  7,  3,  ..., 11, 16,  6])

In [35]:
len(labels)

8012

In [36]:
# idx_train = [i for i in range(features.shape[0])]
idx_train = train_obs
idx_test = test_obs

In [37]:
labels_train = labels[idx_train]

In [38]:
features = torch.FloatTensor(adata.X.toarray())
adj = (adata.obsp['distances']>0)*1

In [39]:
adj

<8012x8012 sparse matrix of type '<class 'numpy.int64'>'
	with 72108 stored elements in Compressed Sparse Row format>

In [43]:
perturbed_adj = torch.load(f"Metattack_xenium_neig{num_neig}.pickle")

In [44]:
perturbed_adj

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [45]:
model = GCNSVD(nfeat=features.shape[1],
          nhid=16,
          nclass=labels.max().item() + 1,
          dropout=0.5, device='cpu').to('cpu')
model.fit(features, perturbed_adj.cpu(), labels, idx_train, k=20)

=== GCN-SVD: rank=20 ===
rank_before = 20
rank_after = 20
Epoch 0, training loss: 2.9727909564971924
Epoch 10, training loss: 2.182591438293457
Epoch 20, training loss: 1.9567217826843262
Epoch 30, training loss: 1.7908519506454468
Epoch 40, training loss: 1.7047377824783325
Epoch 50, training loss: 1.6293301582336426
Epoch 60, training loss: 1.5895817279815674
Epoch 70, training loss: 1.5354822874069214
Epoch 80, training loss: 1.5070204734802246
Epoch 90, training loss: 1.4909987449645996
Epoch 100, training loss: 1.4637699127197266
Epoch 110, training loss: 1.447478175163269
Epoch 120, training loss: 1.4223519563674927
Epoch 130, training loss: 1.419156789779663
Epoch 140, training loss: 1.4040783643722534
Epoch 150, training loss: 1.3898926973342896
Epoch 160, training loss: 1.3799371719360352
Epoch 170, training loss: 1.3710711002349854
Epoch 180, training loss: 1.3667612075805664
Epoch 190, training loss: 1.358947992324829


In [46]:
model = model.to('cpu')

In [47]:
_,predicted = torch.max(torch.softmax(model.predict(features, perturbed_adj.cpu()), dim=1), 1)

=== GCN-SVD: rank=20 ===
rank_before = 20
rank_after = 20


In [48]:
len(labels)

8012

In [49]:
len(predicted.cpu().numpy())

8012

In [50]:
print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

              precision    recall  f1-score   support

           0       0.70      0.56      0.62       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.55      0.81      0.66       634
           4       0.89      0.27      0.41       245
           5       0.74      0.76      0.75       446
           6       0.89      0.92      0.90      1263
           7       0.54      0.82      0.65       851
           8       0.00      0.00      0.00        20
           9       0.98      0.42      0.59       100
          10       0.00      0.00      0.00         8
          11       0.66      0.61      0.64       415
          12       0.00      0.00      0.00        11
          13       0.90      0.34      0.50       125
          14       0.81      0.79      0.80       467
          15       0.00      0.00      0.00         7
          16       0.79      0.57      0.66       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [ ]:

for num_neig in [20,30,40,50]:
#     num_neig = 10
    print(num_neig)
    sc.pp.neighbors(adata, use_rep = 'spatial', n_neighbors=num_neig)
    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['celltype'])).long()

    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())
    adj = (adata.obsp['distances']>0)*1

    adj

    perturbed_adj = torch.load(f"Metattack_xenium_neig{num_neig}.pickle")

    perturbed_adj

    model = GCNSVD(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device='cpu').to('cpu')
    model.fit(features, perturbed_adj.cpu(), labels, idx_train, k=20)

    model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(features, perturbed_adj.cpu()), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))



20
=== GCN-SVD: rank=20 ===


In [54]:
# train_edges
for num_neig in [10,20,30,40,50]:
#     num_neig = 50
    print(num_neig)

    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['celltype'])).long()

    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())
    adj = (adata.obsp['distances']>0)*1

    adj

    perturbed_adj = torch.load(f"Metattack_xenium_neig{num_neig}.pickle")

    perturbed_adj

    model = GCNJaccard(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device='cpu').to('cpu')
    print(model)
    model.fit(features.numpy(), perturbed_adj.cpu().numpy(), labels, idx_train, threshold=0.03)

    model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(features.numpy(), perturbed_adj.cpu().numpy()), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

10
GCNJaccard(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
)
removed 27 edges in the original graph
Epoch 0, training loss: 3.0334949493408203
Epoch 10, training loss: 1.892798662185669


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/deeprobust/graph/utils.py:356: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:618.)
  return torch.sparse.FloatTensor(sparseconcat.t(),sparsedata,torch.Size(sparse_mx.shape))


Epoch 20, training loss: 1.6789863109588623
Epoch 30, training loss: 1.593551516532898
Epoch 40, training loss: 1.533368706703186
Epoch 50, training loss: 1.4963281154632568
Epoch 60, training loss: 1.460618019104004
Epoch 70, training loss: 1.4432920217514038
Epoch 80, training loss: 1.4221781492233276
Epoch 90, training loss: 1.4011298418045044
Epoch 100, training loss: 1.3778668642044067
Epoch 110, training loss: 1.3702690601348877
Epoch 120, training loss: 1.3504090309143066
Epoch 130, training loss: 1.3370481729507446
Epoch 140, training loss: 1.3249099254608154
Epoch 150, training loss: 1.3151485919952393
Epoch 160, training loss: 1.297070860862732
Epoch 170, training loss: 1.2982299327850342
Epoch 180, training loss: 1.2815113067626953
Epoch 190, training loss: 1.276042103767395
removed 27 edges in the original graph
              precision    recall  f1-score   support

           0       0.38      0.23      0.29       187
           1       0.40      0.02      0.04        94
 

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

GCNJaccard(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
)
removed 61 edges in the original graph
Epoch 0, training loss: 2.9773483276367188
Epoch 10, training loss: 2.0912463665008545
Epoch 20, training loss: 1.848392367362976
Epoch 30, training loss: 1.7446143627166748
Epoch 40, training loss: 1.6817872524261475
Epoch 50, training loss: 1.6495364904403687
Epoch 60, training loss: 1.6342729330062866
Epoch 70, training loss: 1.6084108352661133
Epoch 80, training loss: 1.5979162454605103
Epoch 90, training loss: 1.5881627798080444
Epoch 100, training loss: 1.586423397064209
Epoch 110, training loss: 1.5656647682189941
Epoch 120, training loss: 1.561522364616394
Epoch 130, training loss: 1.5490479469299316
Epoch 140, training loss: 1.5427322387695312
Epoch 150, training loss: 1.5439860820770264
Epoch 160, training loss: 1.5327457189559937
Epoch 170, training loss: 1.5255815982818604
Epoch 180, training loss: 1.527014970779419
Epoch 190, training loss: 1.5279

...

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

GCNJaccard(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
)
removed 154 edges in the original graph
Epoch 0, training loss: 2.9358603954315186
Epoch 10, training loss: 2.109574317932129
Epoch 20, training loss: 1.9047893285751343
Epoch 30, training loss: 1.7912102937698364
Epoch 40, training loss: 1.7459160089492798
Epoch 50, training loss: 1.7106857299804688
Epoch 60, training loss: 1.6935533285140991
Epoch 70, training loss: 1.682249903678894
Epoch 80, training loss: 1.6835068464279175
Epoch 90, training loss: 1.6673318147659302
Epoch 100, training loss: 1.6589361429214478
Epoch 110, training loss: 1.6557402610778809
Epoch 120, training loss: 1.6499884128570557
Epoch 130, training loss: 1.6452980041503906
Epoch 140, training loss: 1.6461782455444336
Epoch 150, training loss: 1.6349443197250366
Epoch 160, training loss: 1.6277010440826416
Epoch 170, training loss: 1.630913257598877
Epoch 180, training loss: 1.6274884939193726
Epoch 190, training loss: 1.61

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

GCNJaccard(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
)
removed 195 edges in the original graph
Epoch 0, training loss: 2.925208568572998
Epoch 10, training loss: 2.0665149688720703
Epoch 20, training loss: 1.9160100221633911
Epoch 30, training loss: 1.8184146881103516
Epoch 40, training loss: 1.7705813646316528
Epoch 50, training loss: 1.7472808361053467
Epoch 60, training loss: 1.7302509546279907
Epoch 70, training loss: 1.7210168838500977
Epoch 80, training loss: 1.7144907712936401
Epoch 90, training loss: 1.7014036178588867
Epoch 100, training loss: 1.696903944015503
Epoch 110, training loss: 1.6893857717514038
Epoch 120, training loss: 1.6878329515457153
Epoch 130, training loss: 1.6821057796478271
Epoch 140, training loss: 1.6741390228271484
Epoch 150, training loss: 1.671608328819275
Epoch 160, training loss: 1.6646555662155151
Epoch 170, training loss: 1.662613868713379
Epoch 180, training loss: 1.655388355255127
Epoch 190, training loss: 1.6559

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [9]:
# train_edges
for num_neig in [10,20,30,40,50]:
#     num_neig = 50
    print(num_neig)

    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['cluster'])).long()

    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())


    perturbed_adj = torch.load(f"Metattack_xenium_neig{num_neig}.pickle")


    model = RGCN(nnodes=perturbed_adj.shape[0], nfeat=features.shape[1],
                 nclass=labels.max()+1, nhid=32, device='cpu')
    model.fit(features, perturbed_adj.cpu(), labels, idx_train, train_iters=200)

    model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

10
=== training rgcn model ===
Epoch 0, training loss: 25.28010368347168
Epoch 10, training loss: nan
Epoch 20, training loss: nan
Epoch 30, training loss: nan
Epoch 40, training loss: nan
Epoch 50, training loss: nan
Epoch 60, training loss: nan
Epoch 70, training loss: nan
Epoch 80, training loss: nan
Epoch 90, training loss: nan
Epoch 100, training loss: nan
Epoch 110, training loss: nan
Epoch 120, training loss: nan
Epoch 130, training loss: nan
Epoch 140, training loss: nan
Epoch 150, training loss: nan
Epoch 160, training loss: nan
Epoch 170, training loss: nan
Epoch 180, training loss: nan
Epoch 190, training loss: nan


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.03      1.00      0.07       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.00      0.00      0.00       634
           4       0.00      0.00      0.00       245
           5       0.00      0.00      0.00       446
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.00      0.00      0.00       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.03      1.00      0.07       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.00      0.00      0.00       634
           4       0.00      0.00      0.00       245
           5       0.00      0.00      0.00       446
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.00      0.00      0.00       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.03      1.00      0.07       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.00      0.00      0.00       634
           4       0.00      0.00      0.00       245
           5       0.00      0.00      0.00       446
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.00      0.00      0.00       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.03      1.00      0.07       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.00      0.00      0.00       634
           4       0.00      0.00      0.00       245
           5       0.00      0.00      0.00       446
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.00      0.00      0.00       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [11]:
# train_edges
for num_neig in [10,20,30,40,50]:
#     num_neig = 50
    print(num_neig)

    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['cluster'])).long()

    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())


    perturbed_adj = torch.load(f"Metattack_xenium_neig{num_neig}.pickle")


    model = RGCN(nnodes=perturbed_adj.shape[0], nfeat=features.shape[1],
                 nclass=labels.max()+1, nhid=32, device='cpu')
    model.fit(features, perturbed_adj.cpu(), labels, idx_train, train_iters=200)

    model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

10
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
RGCN(
  (gc1): GGCL_F()
  (gc2): GGCL_D()
)
=== training rgcn model ===
Epoch 0, training loss: 28.440462112426758
Epoch 10, training loss: nan
Epoch 20, training loss: nan
Epoch 30, training loss: nan
Epoch 40, training loss: nan
Epoch 50, training loss: nan
Epoch 60, training loss: nan
Epoch 70, training loss: nan
Epoch 80, training loss: nan
Epoch 90, training loss: nan
Epoch 100, training loss: nan
Epoch 110, training loss: nan
Epoch 120, training loss: nan
Epoch 130, training loss: nan
Epoch 140, training loss: nan
Epoch 150, training loss: nan
Epoch 160, training loss: nan
Epoch 170, training loss: nan
Epoch 180, training loss: nan
Epoch 190, training loss: nan


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.03      1.00      0.07       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.00      0.00      0.00       634
           4       0.00      0.00      0.00       245
           5       0.00      0.00      0.00       446
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.00      0.00      0.00       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.03      1.00      0.07       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.00      0.00      0.00       634
           4       0.00      0.00      0.00       245
           5       0.00      0.00      0.00       446
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.00      0.00      0.00       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.03      1.00      0.07       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.00      0.00      0.00       634
           4       0.00      0.00      0.00       245
           5       0.00      0.00      0.00       446
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.00      0.00      0.00       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.03      1.00      0.07       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.00      0.00      0.00       634
           4       0.00      0.00      0.00       245
           5       0.00      0.00      0.00       446
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.00      0.00      0.00       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [13]:
# train_edges
for num_neig in [10,20,30,40,50]:
#     num_neig = 50
    print(num_neig)

    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['cluster'])).long()
    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())


    perturbed_adj = torch.load(f"Metattack_xenium_neig{num_neig}.pickle")

    perturbed_adj

    model = SimPGCN(nnodes=features.shape[0], nfeat=features.shape[1],
    nhid=16, nclass=int(labels.max()+1), device='cuda')
    model = model.to('cuda')
    print(model)
    model.fit(features.cuda(), perturbed_adj.cuda(), labels, idx_train, train_iters=200)

#     model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

10


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/deeprobust/graph/utils.py:356: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:618.)
  return torch.sparse.FloatTensor(sparseconcat.t(),sparsedata,torch.Size(sparse_mx.shape))


SimPGCN(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
  (scores): ParameterList(
      (0): Parameter containing: [torch.float32 of size 313x1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 16x1 (cuda:0)]
  )
  (bias): ParameterList(
      (0): Parameter containing: [torch.float32 of size 1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 1 (cuda:0)]
  )
  (D_k): ParameterList(
      (0): Parameter containing: [torch.float32 of size 313x1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 16x1 (cuda:0)]
  )
  (D_bias): ParameterList(
      (0): Parameter containing: [torch.float32 of size 1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 1 (cuda:0)]
  )
  (linear): Linear(in_features=16, out_features=1, bias=True)
)
loading saved_knn/cosine_sims_(8012, 313).npy
number of sampled: 76212


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.75      0.89      0.81       187
           1       0.69      0.31      0.43        94
           2       0.61      0.43      0.50       193
           3       0.77      0.90      0.83       634
           4       0.78      0.53      0.63       245
           5       0.77      0.65      0.70       446
           6       0.84      0.95      0.89      1263
           7       0.63      0.78      0.70       851
           8       0.00      0.00      0.00        20
           9       0.94      0.51      0.66       100
          10       0.00      0.00      0.00         8
          11       0.75      0.78      0.76       415
          12       0.00      0.00      0.00        11
          13       0.90      0.82      0.86       125
          14       0.81      0.87      0.84       467
          15       0.00      0.00      0.00         7
          16       0.93      0.29      0.45       300
          17       0.00    

...

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.92      0.95      0.93       187
           1       0.90      0.81      0.85        94
           2       0.74      0.64      0.68       193
           3       0.88      0.92      0.90       634
           4       0.83      0.82      0.82       245
           5       0.78      0.74      0.76       446
           6       0.88      0.95      0.92      1263
           7       0.84      0.80      0.82       851
           8       1.00      0.65      0.79        20
           9       0.93      0.90      0.91       100
          10       0.00      0.00      0.00         8
          11       0.78      0.83      0.80       415
          12       0.00      0.00      0.00        11
          13       0.91      0.90      0.91       125
          14       0.87      0.87      0.87       467
          15       0.00      0.00      0.00         7
          16       0.91      0.90      0.90       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.89      0.95      0.91       187
           1       0.79      0.52      0.63        94
           2       0.71      0.65      0.68       193
           3       0.85      0.91      0.88       634
           4       0.81      0.74      0.78       245
           5       0.80      0.79      0.79       446
           6       0.88      0.95      0.91      1263
           7       0.84      0.78      0.81       851
           8       1.00      0.70      0.82        20
           9       0.93      0.89      0.91       100
          10       0.00      0.00      0.00         8
          11       0.73      0.80      0.77       415
          12       0.00      0.00      0.00        11
          13       0.90      0.90      0.90       125
          14       0.89      0.87      0.88       467
          15       0.00      0.00      0.00         7
          16       0.91      0.89      0.90       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [16]:

for num_neig in [10, 20,30,40,50]:
#     num_neig = 10
    print(num_neig)
    sc.pp.neighbors(adata, use_rep = 'spatial', n_neighbors=num_neig)
    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['celltype'])).long()

    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())

    perturbed_adj = torch.load(f"DICE_xenium_neig{num_neig}.pickle")

    perturbed_adj

    model = GCNSVD(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device='cpu').to('cpu')
    model.fit(features, perturbed_adj, labels, idx_train, k=20)

    model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(features, perturbed_adj), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))



10
=== GCN-SVD: rank=20 ===
rank_after = 20
Epoch 0, training loss: 3.1911470890045166
Epoch 10, training loss: 1.8007349967956543
Epoch 20, training loss: 1.4469114542007446
Epoch 30, training loss: 1.2789283990859985
Epoch 40, training loss: 1.163910150527954
Epoch 50, training loss: 1.114203691482544
Epoch 60, training loss: 1.069741129875183
Epoch 70, training loss: 1.0350055694580078
Epoch 80, training loss: 1.002153754234314
Epoch 90, training loss: 0.9653157591819763
Epoch 100, training loss: 0.963822066783905
Epoch 110, training loss: 0.9459049105644226
Epoch 120, training loss: 0.9260398745536804
Epoch 130, training loss: 0.9067915081977844
Epoch 140, training loss: 0.8919119238853455
Epoch 150, training loss: 0.9010905623435974
Epoch 160, training loss: 0.8735173940658569
Epoch 170, training loss: 0.8680386543273926
Epoch 180, training loss: 0.8626048564910889
Epoch 190, training loss: 0.8612006902694702
=== GCN-SVD: rank=20 ===
rank_after = 20


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.86      0.81      0.84       187
           1       0.95      0.63      0.76        94
           2       0.89      0.47      0.62       193
           3       0.79      0.93      0.86       634
           4       0.92      0.82      0.87       245
           5       0.93      0.90      0.92       446
           6       0.93      0.99      0.96      1263
           7       0.80      0.86      0.83       851
           8       0.00      0.00      0.00        20
           9       0.99      0.82      0.90       100
          10       0.00      0.00      0.00         8
          11       0.82      0.83      0.82       415
          12       0.00      0.00      0.00        11
          13       0.93      0.88      0.91       125
          14       0.87      0.93      0.90       467
          15       0.00      0.00      0.00         7
          16       0.98      0.84      0.90       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.31      0.09      0.13       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.36      0.62      0.46       634
           4       0.92      0.10      0.18       245
           5       0.47      0.48      0.47       446
           6       0.78      0.95      0.85      1263
           7       0.45      0.53      0.49       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.77      0.25      0.38       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.45      0.93      0.61       467
          15       0.00      0.00      0.00         7
          16       0.97      0.30      0.45       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.25      0.06      0.10       187
           1       0.00      0.00      0.00        94
           2       1.00      0.01      0.01       193
           3       0.32      0.59      0.42       634
           4       0.96      0.09      0.16       245
           5       0.42      0.30      0.35       446
           6       0.66      0.94      0.77      1263
           7       0.38      0.41      0.39       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.81      0.16      0.27       415
          12       0.00      0.00      0.00        11
          13       1.00      0.04      0.08       125
          14       0.41      0.82      0.54       467
          15       0.00      0.00      0.00         7
          16       0.93      0.19      0.31       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.30      0.37      0.33       634
           4       0.00      0.00      0.00       245
           5       0.31      0.25      0.27       446
           6       0.54      0.92      0.68      1263
           7       0.31      0.36      0.33       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.40      0.91      0.55       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [17]:
# train_edges
for num_neig in [10,20,30,40,50]:
#     num_neig = 50
    print(num_neig)

    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['cluster'])).long()

    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())


    perturbed_adj = torch.load(f"DICE_xenium_neig{num_neig}.pickle")
    model = GCNJaccard(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device='cpu').to('cpu')
    print(model)
    model.fit(features.numpy(), perturbed_adj, labels, idx_train, threshold=0.03)

    model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(features.numpy(), perturbed_adj), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

10
GCNJaccard(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
)
removed 25 edges in the original graph
Epoch 0, training loss: 3.0414927005767822
Epoch 10, training loss: 1.8473342657089233
Epoch 20, training loss: 1.6245815753936768
Epoch 30, training loss: 1.524501919746399
Epoch 40, training loss: 1.46983802318573
Epoch 50, training loss: 1.4319013357162476
Epoch 60, training loss: 1.4007295370101929
Epoch 70, training loss: 1.386719822883606
Epoch 80, training loss: 1.356926679611206
Epoch 90, training loss: 1.3326427936553955
Epoch 100, training loss: 1.3163522481918335
Epoch 110, training loss: 1.3145718574523926
Epoch 120, training loss: 1.2971503734588623
Epoch 130, training loss: 1.2927781343460083
Epoch 140, training loss: 1.2766680717468262
Epoch 150, training loss: 1.2667977809906006
Epoch 160, training loss: 1.2539204359054565
Epoch 170, training loss: 1.2463165521621704
Epoch 180, training loss: 1.246575117111206
Epoch 190, training loss: 1.242

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

Epoch 0, training loss: 3.127209186553955
Epoch 10, training loss: 1.979985237121582
Epoch 20, training loss: 1.7823646068572998
Epoch 30, training loss: 1.6796871423721313
Epoch 40, training loss: 1.6121819019317627
Epoch 50, training loss: 1.583609938621521
Epoch 60, training loss: 1.5529505014419556
Epoch 70, training loss: 1.5363656282424927
Epoch 80, training loss: 1.5207018852233887
Epoch 90, training loss: 1.5150691270828247
Epoch 100, training loss: 1.4933974742889404
Epoch 110, training loss: 1.488590121269226
Epoch 120, training loss: 1.4750254154205322
Epoch 130, training loss: 1.4689923524856567
Epoch 140, training loss: 1.4648163318634033
Epoch 150, training loss: 1.4584487676620483
Epoch 160, training loss: 1.4535865783691406
Epoch 170, training loss: 1.436596155166626
Epoch 180, training loss: 1.4362802505493164
Epoch 190, training loss: 1.4317505359649658
removed 51 edges in the original graph
              precision    recall  f1-score   support

           0       0.2

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

removed 91 edges in the original graph
Epoch 0, training loss: 2.92002272605896
Epoch 10, training loss: 2.0558760166168213
Epoch 20, training loss: 1.859780192375183
Epoch 30, training loss: 1.7576695680618286
Epoch 40, training loss: 1.6893984079360962
Epoch 50, training loss: 1.6560885906219482
Epoch 60, training loss: 1.6278427839279175
Epoch 70, training loss: 1.614460825920105
Epoch 80, training loss: 1.594074010848999
Epoch 90, training loss: 1.5819915533065796
Epoch 100, training loss: 1.569214940071106
Epoch 110, training loss: 1.5518238544464111
Epoch 120, training loss: 1.5471506118774414
Epoch 130, training loss: 1.5330188274383545
Epoch 140, training loss: 1.5279734134674072
Epoch 150, training loss: 1.51753830909729
Epoch 160, training loss: 1.516008734703064
Epoch 170, training loss: 1.5152090787887573
Epoch 180, training loss: 1.5055267810821533
Epoch 190, training loss: 1.501679539680481
removed 91 edges in the original graph
              precision    recall  f1-score

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

removed 123 edges in the original graph
Epoch 0, training loss: 3.4952266216278076
Epoch 10, training loss: 2.2030813694000244
Epoch 20, training loss: 1.933435082435608
Epoch 30, training loss: 1.8087925910949707
Epoch 40, training loss: 1.735161304473877
Epoch 50, training loss: 1.70942223072052
Epoch 60, training loss: 1.6880568265914917
Epoch 70, training loss: 1.6716744899749756
Epoch 80, training loss: 1.6604013442993164
Epoch 90, training loss: 1.6510653495788574
Epoch 100, training loss: 1.646320104598999
Epoch 110, training loss: 1.6345561742782593
Epoch 120, training loss: 1.6292988061904907
Epoch 130, training loss: 1.6224478483200073
Epoch 140, training loss: 1.6222654581069946
Epoch 150, training loss: 1.6109213829040527
Epoch 160, training loss: 1.6093406677246094
Epoch 170, training loss: 1.6052156686782837
Epoch 180, training loss: 1.6058300733566284
Epoch 190, training loss: 1.600394368171692
removed 123 edges in the original graph


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.20      0.06      0.10       187
           1       0.00      0.00      0.00        94
           2       0.67      0.02      0.04       193
           3       0.37      0.62      0.47       634
           4       0.00      0.00      0.00       245
           5       0.53      0.67      0.59       446
           6       0.66      0.91      0.77      1263
           7       0.38      0.46      0.41       851
           8       0.00      0.00      0.00        20
           9       0.35      0.09      0.14       100
          10       0.00      0.00      0.00         8
          11       0.27      0.06      0.10       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.55      0.89      0.68       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [19]:
# train_edges
for num_neig in [10,20,30,40,50]:
#     num_neig = 50
    print(num_neig)

    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['cluster'])).long()

    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())


    perturbed_adj = torch.load(f"DICE_xenium_neig{num_neig}.pickle")
    
    perturbed_adj = torch.FloatTensor(perturbed_adj.todense())

    model = RGCN(nnodes=perturbed_adj.shape[0], nfeat=features.shape[1],
                 nclass=labels.max()+1, nhid=32, device='cpu')
    model.fit(features, perturbed_adj, labels, idx_train, train_iters=200)

    model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

10
=== training rgcn model ===
Epoch 0, training loss: 28.84331703186035
Epoch 10, training loss: 9.99221134185791
Epoch 20, training loss: 9.286331176757812
Epoch 30, training loss: 7.550395965576172
Epoch 40, training loss: 6.654519557952881
Epoch 50, training loss: 6.116572856903076
Epoch 60, training loss: 5.72416877746582
Epoch 70, training loss: 5.479998588562012
Epoch 80, training loss: 5.2467145919799805
Epoch 90, training loss: 5.204430103302002
Epoch 100, training loss: 5.073188781738281
Epoch 110, training loss: 5.081057071685791
Epoch 120, training loss: 5.016446590423584
Epoch 130, training loss: 4.934828758239746
Epoch 140, training loss: 4.828463554382324
Epoch 150, training loss: 4.768413543701172
Epoch 160, training loss: 4.793855667114258
Epoch 170, training loss: 4.766519069671631
Epoch 180, training loss: 4.7619829177856445
Epoch 190, training loss: 4.746726036071777


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.07      0.01      0.01       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.40      0.70      0.51       634
           4       0.00      0.00      0.00       245
           5       0.49      0.80      0.61       446
           6       0.67      0.94      0.78      1263
           7       0.39      0.44      0.41       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.56      0.92      0.69       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.42      0.48      0.45       634
           4       0.00      0.00      0.00       245
           5       0.44      0.77      0.56       446
           6       0.63      0.94      0.75      1263
           7       0.32      0.44      0.37       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.53      0.91      0.67       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.33      0.60      0.43       634
           4       0.00      0.00      0.00       245
           5       0.46      0.67      0.54       446
           6       0.63      0.93      0.75      1263
           7       0.30      0.32      0.31       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.52      0.89      0.66       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.33      0.54      0.41       634
           4       0.00      0.00      0.00       245
           5       0.54      0.48      0.50       446
           6       0.60      0.94      0.73      1263
           7       0.29      0.45      0.35       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.57      0.82      0.67       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [20]:
# train_edges
for num_neig in [10,20,30,40,50]:
#     num_neig = 50
    print(num_neig)

    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['cluster'])).long()
    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())

    perturbed_adj = torch.load(f"DICE_xenium_neig{num_neig}.pickle")

    perturbed_adj = torch.FloatTensor(perturbed_adj.todense())

    model = SimPGCN(nnodes=features.shape[0], nfeat=features.shape[1],
    nhid=16, nclass=int(labels.max()+1), device='cuda')
    model = model.to('cuda')
    print(model)
    model.fit(features.cuda(), perturbed_adj, labels, idx_train, train_iters=200)

#     model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

10
SimPGCN(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
  (scores): ParameterList(
      (0): Parameter containing: [torch.float32 of size 313x1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 16x1 (cuda:0)]
  )
  (bias): ParameterList(
      (0): Parameter containing: [torch.float32 of size 1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 1 (cuda:0)]
  )
  (D_k): ParameterList(
      (0): Parameter containing: [torch.float32 of size 313x1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 16x1 (cuda:0)]
  )
  (D_bias): ParameterList(
      (0): Parameter containing: [torch.float32 of size 1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 1 (cuda:0)]
  )
  (linear): Linear(in_features=16, out_features=1, bias=True)
)
loading saved_knn/knn_graph_(8012, 313).npz...
loading saved_knn/cosine_sims_(8012, 313).npy
loading saved_knn/attrsim_sampled_idx_(8012, 313).npy
number of sampled: 76212


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       187
           1       0.88      0.82      0.85        94
           2       0.76      0.76      0.76       193
           3       0.88      0.93      0.90       634
           4       0.84      0.78      0.81       245
           5       0.87      0.85      0.86       446
           6       0.92      0.96      0.94      1263
           7       0.87      0.83      0.85       851
           8       1.00      0.65      0.79        20
           9       0.96      0.87      0.91       100
          10       1.00      0.25      0.40         8
          11       0.81      0.81      0.81       415
          12       1.00      0.09      0.17        11
          13       0.90      0.90      0.90       125
          14       0.92      0.93      0.93       467
          15       1.00      0.14      0.25         7
          16       0.93      0.91      0.92       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.89      0.94      0.91       187
           1       0.87      0.78      0.82        94
           2       0.74      0.64      0.69       193
           3       0.87      0.92      0.89       634
           4       0.81      0.82      0.82       245
           5       0.85      0.77      0.81       446
           6       0.88      0.95      0.91      1263
           7       0.82      0.80      0.81       851
           8       1.00      0.55      0.71        20
           9       0.89      0.87      0.88       100
          10       0.00      0.00      0.00         8
          11       0.81      0.79      0.80       415
          12       0.00      0.00      0.00        11
          13       0.91      0.88      0.89       125
          14       0.88      0.89      0.88       467
          15       0.00      0.00      0.00         7
          16       0.92      0.89      0.91       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.90      0.95      0.92       187
           1       0.88      0.78      0.82        94
           2       0.68      0.68      0.68       193
           3       0.87      0.92      0.89       634
           4       0.84      0.77      0.80       245
           5       0.83      0.82      0.83       446
           6       0.90      0.96      0.93      1263
           7       0.84      0.81      0.83       851
           8       1.00      0.55      0.71        20
           9       0.97      0.83      0.89       100
          10       0.00      0.00      0.00         8
          11       0.81      0.79      0.80       415
          12       0.00      0.00      0.00        11
          13       0.93      0.87      0.90       125
          14       0.88      0.90      0.89       467
          15       0.00      0.00      0.00         7
          16       0.91      0.89      0.90       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.92      0.95      0.93       187
           1       0.85      0.71      0.77        94
           2       0.74      0.71      0.73       193
           3       0.88      0.92      0.90       634
           4       0.83      0.84      0.84       245
           5       0.82      0.81      0.81       446
           6       0.90      0.96      0.93      1263
           7       0.86      0.81      0.84       851
           8       0.91      0.50      0.65        20
           9       0.98      0.84      0.90       100
          10       0.00      0.00      0.00         8
          11       0.78      0.80      0.79       415
          12       0.00      0.00      0.00        11
          13       0.90      0.91      0.90       125
          14       0.89      0.91      0.90       467
          15       0.00      0.00      0.00         7
          16       0.92      0.92      0.92       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [21]:

for num_neig in [10, 20,30,40,50]:
#     num_neig = 10
    print(num_neig)
    sc.pp.neighbors(adata, use_rep = 'spatial', n_neighbors=num_neig)
    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['celltype'])).long()

    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())


    perturbed_adj = torch.load(f"MinMax_xenium_neig{num_neig}.pickle")

    perturbed_adj

    model = GCNSVD(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device='cpu').to('cpu')
    model.fit(features, perturbed_adj.cpu(), labels, idx_train, k=20)

    model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(features, perturbed_adj.cpu()), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))



10
=== GCN-SVD: rank=20 ===
rank_before = 20
rank_after = 20
Epoch 0, training loss: 3.562967538833618
Epoch 10, training loss: 1.878933310508728
Epoch 20, training loss: 1.4915456771850586
Epoch 30, training loss: 1.3195672035217285
Epoch 40, training loss: 1.2053616046905518
Epoch 50, training loss: 1.126868486404419
Epoch 60, training loss: 1.0666207075119019
Epoch 70, training loss: 1.049060583114624
Epoch 80, training loss: 1.012221097946167
Epoch 90, training loss: 0.9876600503921509
Epoch 100, training loss: 0.9656801223754883
Epoch 110, training loss: 0.9618097543716431
Epoch 120, training loss: 0.9202957153320312
Epoch 130, training loss: 0.9331825375556946
Epoch 140, training loss: 0.9086806774139404
Epoch 150, training loss: 0.898337185382843
Epoch 160, training loss: 0.866692304611206
Epoch 170, training loss: 0.8718854784965515
Epoch 180, training loss: 0.8674840331077576
Epoch 190, training loss: 0.8717113137245178
=== GCN-SVD: rank=20 ===
rank_before = 20
rank_after = 20

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.90      0.81      0.85       187
           1       0.97      0.60      0.74        94
           2       0.95      0.56      0.71       193
           3       0.81      0.93      0.87       634
           4       0.97      0.79      0.87       245
           5       0.95      0.89      0.92       446
           6       0.94      0.99      0.96      1263
           7       0.79      0.88      0.83       851
           8       0.00      0.00      0.00        20
           9       0.97      0.84      0.90       100
          10       0.00      0.00      0.00         8
          11       0.79      0.84      0.81       415
          12       0.00      0.00      0.00        11
          13       0.92      0.86      0.89       125
          14       0.85      0.93      0.89       467
          15       0.00      0.00      0.00         7
          16       0.98      0.81      0.89       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.31      0.10      0.15       187
           1       1.00      0.01      0.02        94
           2       0.00      0.00      0.00       193
           3       0.36      0.66      0.46       634
           4       0.98      0.17      0.29       245
           5       0.53      0.47      0.50       446
           6       0.80      0.94      0.87      1263
           7       0.42      0.57      0.49       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.81      0.26      0.40       415
          12       0.00      0.00      0.00        11
          13       1.00      0.11      0.20       125
          14       0.46      0.79      0.58       467
          15       0.00      0.00      0.00         7
          16       0.87      0.37      0.52       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.31      0.53      0.39       634
           4       0.00      0.00      0.00       245
           5       0.44      0.35      0.39       446
           6       0.69      0.92      0.79      1263
           7       0.33      0.45      0.38       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.92      0.14      0.25       415
          12       0.00      0.00      0.00        11
          13       1.00      0.05      0.09       125
          14       0.40      0.82      0.54       467
          15       0.00      0.00      0.00         7
          16       0.83      0.20      0.32       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.29      0.42      0.34       634
           4       0.00      0.00      0.00       245
           5       0.31      0.24      0.27       446
           6       0.53      0.92      0.67      1263
           7       0.32      0.31      0.32       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.40      0.91      0.56       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [10]:
# train_edges
for num_neig in [10,20,30,40,50]:
#     num_neig = 50
    print(num_neig)

    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['celltype'])).long()

    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())

    perturbed_adj = torch.load(f"MinMax_xenium_neig{num_neig}.pickle")

    perturbed_adj

    model = GCNJaccard(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device='cpu').to('cpu')
    print(model)
    model.fit(features.numpy(), perturbed_adj.cpu().numpy(), labels, idx_train, threshold=0.03)

    model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(features.numpy(), perturbed_adj.cpu().numpy()), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

10
GCNJaccard(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
)
removed 25 edges in the original graph
Epoch 0, training loss: 3.2479867935180664
Epoch 10, training loss: 1.9432103633880615


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/deeprobust/graph/utils.py:356: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:618.)
  return torch.sparse.FloatTensor(sparseconcat.t(),sparsedata,torch.Size(sparse_mx.shape))


Epoch 20, training loss: 1.682207465171814
Epoch 30, training loss: 1.5790047645568848
Epoch 40, training loss: 1.5355548858642578
Epoch 50, training loss: 1.5091086626052856
Epoch 60, training loss: 1.486201286315918
Epoch 70, training loss: 1.4549912214279175
Epoch 80, training loss: 1.441899299621582
Epoch 90, training loss: 1.4234689474105835
Epoch 100, training loss: 1.41800856590271
Epoch 110, training loss: 1.3864325284957886
Epoch 120, training loss: 1.382501482963562
Epoch 130, training loss: 1.373105764389038
Epoch 140, training loss: 1.360783338546753
Epoch 150, training loss: 1.3501249551773071
Epoch 160, training loss: 1.3370567560195923
Epoch 170, training loss: 1.3244481086730957
Epoch 180, training loss: 1.324769139289856
Epoch 190, training loss: 1.310683012008667
removed 25 edges in the original graph
              precision    recall  f1-score   support

           0       0.39      0.24      0.29       187
           1       0.42      0.15      0.22        94
      

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

GCNJaccard(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
)
removed 51 edges in the original graph
Epoch 0, training loss: 3.325204849243164
Epoch 10, training loss: 2.0615689754486084
Epoch 20, training loss: 1.8728567361831665
Epoch 30, training loss: 1.7692114114761353
Epoch 40, training loss: 1.7031574249267578
Epoch 50, training loss: 1.6526767015457153
Epoch 60, training loss: 1.6251296997070312
Epoch 70, training loss: 1.605211853981018
Epoch 80, training loss: 1.5955828428268433
Epoch 90, training loss: 1.5819300413131714
Epoch 100, training loss: 1.5795722007751465
Epoch 110, training loss: 1.564640998840332
Epoch 120, training loss: 1.5548839569091797
Epoch 130, training loss: 1.545647144317627
Epoch 140, training loss: 1.5407209396362305
Epoch 150, training loss: 1.5403285026550293
Epoch 160, training loss: 1.5242573022842407
Epoch 170, training loss: 1.5073878765106201
Epoch 180, training loss: 1.5081526041030884
Epoch 190, training loss: 1.5019

...

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

GCNJaccard(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
)
removed 123 edges in the original graph
Epoch 0, training loss: 2.93445086479187
Epoch 10, training loss: 2.038065195083618
Epoch 20, training loss: 1.8208682537078857
Epoch 30, training loss: 1.726250171661377
Epoch 40, training loss: 1.6894385814666748
Epoch 50, training loss: 1.6691089868545532
Epoch 60, training loss: 1.6491917371749878
Epoch 70, training loss: 1.6435619592666626
Epoch 80, training loss: 1.633176326751709
Epoch 90, training loss: 1.6262152194976807
Epoch 100, training loss: 1.6163071393966675
Epoch 110, training loss: 1.6063495874404907
Epoch 120, training loss: 1.5968248844146729
Epoch 130, training loss: 1.591834545135498
Epoch 140, training loss: 1.5879511833190918
Epoch 150, training loss: 1.5832767486572266
Epoch 160, training loss: 1.5808007717132568
Epoch 170, training loss: 1.5688711404800415
Epoch 180, training loss: 1.5689506530761719
Epoch 190, training loss: 1.56222

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

GCNJaccard(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
)
removed 163 edges in the original graph
Epoch 0, training loss: 2.865227699279785
Epoch 10, training loss: 2.0093352794647217
Epoch 20, training loss: 1.8411694765090942
Epoch 30, training loss: 1.7662311792373657
Epoch 40, training loss: 1.7144209146499634
Epoch 50, training loss: 1.6971932649612427
Epoch 60, training loss: 1.6857529878616333
Epoch 70, training loss: 1.671379804611206
Epoch 80, training loss: 1.658634066581726
Epoch 90, training loss: 1.6481902599334717
Epoch 100, training loss: 1.642181396484375
Epoch 110, training loss: 1.6418205499649048
Epoch 120, training loss: 1.6354820728302002
Epoch 130, training loss: 1.627368688583374
Epoch 140, training loss: 1.623242974281311
Epoch 150, training loss: 1.6219784021377563
Epoch 160, training loss: 1.6157554388046265
Epoch 170, training loss: 1.6057605743408203
Epoch 180, training loss: 1.6106599569320679
Epoch 190, training loss: 1.60181

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [11]:
# train_edges
for num_neig in [10,20,30,40,50]:
#     num_neig = 50
    print(num_neig)

    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['cluster'])).long()

    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())


    perturbed_adj = torch.load(f"MinMax_xenium_neig{num_neig}.pickle")


    model = RGCN(nnodes=perturbed_adj.shape[0], nfeat=features.shape[1],
                 nclass=labels.max()+1, nhid=32, device='cpu')
    model.fit(features, perturbed_adj.cpu(), labels, idx_train, train_iters=200)

    model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

10
=== training rgcn model ===
Epoch 0, training loss: 27.216615676879883
Epoch 10, training loss: 10.106439590454102
Epoch 20, training loss: 9.107023239135742
Epoch 30, training loss: 7.496109485626221
Epoch 40, training loss: 6.78776741027832
Epoch 50, training loss: 6.237861156463623
Epoch 60, training loss: 5.947803974151611
Epoch 70, training loss: 5.609435081481934
Epoch 80, training loss: 5.405959606170654
Epoch 90, training loss: 5.181933879852295
Epoch 100, training loss: 5.061477184295654
Epoch 110, training loss: 5.047299385070801
Epoch 120, training loss: 5.0080742835998535
Epoch 130, training loss: 4.953723907470703
Epoch 140, training loss: 4.923718452453613
Epoch 150, training loss: 4.877532482147217
Epoch 160, training loss: 4.883557319641113
Epoch 170, training loss: 4.797605991363525
Epoch 180, training loss: 4.757989883422852
Epoch 190, training loss: 4.719488620758057


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.17      0.01      0.02       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.38      0.72      0.50       634
           4       0.00      0.00      0.00       245
           5       0.58      0.67      0.62       446
           6       0.66      0.94      0.78      1263
           7       0.36      0.48      0.41       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.60      0.91      0.72       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.39      0.57      0.46       634
           4       0.00      0.00      0.00       245
           5       0.56      0.60      0.58       446
           6       0.63      0.94      0.75      1263
           7       0.32      0.53      0.40       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.59      0.87      0.71       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.38      0.47      0.42       634
           4       0.00      0.00      0.00       245
           5       0.38      0.81      0.52       446
           6       0.62      0.93      0.74      1263
           7       0.33      0.36      0.35       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.51      0.90      0.65       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00       193
           3       0.33      0.57      0.42       634
           4       0.00      0.00      0.00       245
           5       0.39      0.76      0.51       446
           6       0.64      0.92      0.75      1263
           7       0.35      0.24      0.29       851
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       100
          10       0.00      0.00      0.00         8
          11       0.00      0.00      0.00       415
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00       125
          14       0.44      0.95      0.60       467
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [12]:
# train_edges
for num_neig in [10,20,30,40,50]:
#     num_neig = 50
    print(num_neig)

    le = sklearn.preprocessing.LabelEncoder()

    labels = torch.FloatTensor(le.fit_transform(adata.obs['cluster'])).long()
    labels

    len(labels)

    # idx_train = [i for i in range(features.shape[0])]
    idx_train = train_obs
    idx_test = test_obs

    labels_train = labels[idx_train]

    features = torch.FloatTensor(adata.X.toarray())


    perturbed_adj = torch.load(f"MinMax_xenium_neig{num_neig}.pickle")

    perturbed_adj

    model = SimPGCN(nnodes=features.shape[0], nfeat=features.shape[1],
    nhid=16, nclass=int(labels.max()+1), device='cuda')
    model = model.to('cuda')
    print(model)
    model.fit(features.cuda(), perturbed_adj.cuda(), labels, idx_train, train_iters=200)

#     model = model.to('cpu')

    _,predicted = torch.max(torch.softmax(model.predict(), dim=1), 1)

    len(labels)

    len(predicted.cpu().numpy())

    print(sklearn.metrics.classification_report(labels[train_obs], predicted.cpu().numpy()[train_obs]))

10
SimPGCN(
  (gc1): GraphConvolution (313 -> 16)
  (gc2): GraphConvolution (16 -> 18)
  (scores): ParameterList(
      (0): Parameter containing: [torch.float32 of size 313x1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 16x1 (cuda:0)]
  )
  (bias): ParameterList(
      (0): Parameter containing: [torch.float32 of size 1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 1 (cuda:0)]
  )
  (D_k): ParameterList(
      (0): Parameter containing: [torch.float32 of size 313x1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 16x1 (cuda:0)]
  )
  (D_bias): ParameterList(
      (0): Parameter containing: [torch.float32 of size 1 (cuda:0)]
      (1): Parameter containing: [torch.float32 of size 1 (cuda:0)]
  )
  (linear): Linear(in_features=16, out_features=1, bias=True)
)
loading saved_knn/knn_graph_(8012, 313).npz...
loading saved_knn/cosine_sims_(8012, 313).npy
loading saved_knn/attrsim_sampled_idx_(8012, 313).npy
number of sampled: 76212


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.91      0.94      0.92       187
           1       0.87      0.64      0.74        94
           2       0.69      0.64      0.66       193
           3       0.85      0.90      0.87       634
           4       0.79      0.76      0.77       245
           5       0.82      0.80      0.81       446
           6       0.89      0.95      0.92      1263
           7       0.82      0.81      0.81       851
           8       0.00      0.00      0.00        20
           9       0.95      0.86      0.90       100
          10       0.00      0.00      0.00         8
          11       0.78      0.78      0.78       415
          12       0.00      0.00      0.00        11
          13       0.91      0.88      0.89       125
          14       0.88      0.91      0.90       467
          15       0.00      0.00      0.00         7
          16       0.91      0.88      0.89       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.94      0.91      0.92       187
           1       0.90      0.79      0.84        94
           2       0.78      0.62      0.69       193
           3       0.87      0.91      0.89       634
           4       0.82      0.79      0.81       245
           5       0.77      0.78      0.78       446
           6       0.88      0.96      0.92      1263
           7       0.84      0.82      0.83       851
           8       1.00      0.65      0.79        20
           9       0.96      0.87      0.91       100
          10       1.00      0.25      0.40         8
          11       0.80      0.82      0.81       415
          12       0.00      0.00      0.00        11
          13       0.92      0.88      0.90       125
          14       0.91      0.84      0.87       467
          15       0.00      0.00      0.00         7
          16       0.93      0.95      0.94       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       187
           1       0.91      0.82      0.86        94
           2       0.77      0.65      0.70       193
           3       0.91      0.90      0.90       634
           4       0.80      0.82      0.81       245
           5       0.85      0.81      0.83       446
           6       0.89      0.96      0.93      1263
           7       0.84      0.83      0.84       851
           8       1.00      0.60      0.75        20
           9       0.94      0.89      0.91       100
          10       0.00      0.00      0.00         8
          11       0.77      0.82      0.79       415
          12       0.00      0.00      0.00        11
          13       0.91      0.85      0.88       125
          14       0.91      0.88      0.89       467
          15       0.00      0.00      0.00         7
          16       0.91      0.92      0.91       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

           0       0.87      0.96      0.91       187
           1       0.87      0.62      0.72        94
           2       0.68      0.60      0.63       193
           3       0.86      0.91      0.88       634
           4       0.82      0.78      0.80       245
           5       0.83      0.81      0.82       446
           6       0.90      0.95      0.93      1263
           7       0.82      0.82      0.82       851
           8       1.00      0.55      0.71        20
           9       0.95      0.81      0.88       100
          10       0.00      0.00      0.00         8
          11       0.78      0.78      0.78       415
          12       0.00      0.00      0.00        11
          13       0.92      0.86      0.89       125
          14       0.86      0.91      0.89       467
          15       0.00      0.00      0.00         7
          16       0.92      0.89      0.91       300
          17       0.00    

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control